In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
%matplotlib inline
np.set_printoptions(precision=2)
pd.set_option('precision', 2)
np.set_printoptions(suppress=True)
sns.set(style='whitegrid')

In [2]:
#1. Load training and test datasets
train_path = 'data/train.csv'
test_path = 'data/test.csv'
train_data = pd.read_csv(train_path, index_col='Id')
test_data = pd.read_csv(test_path, index_col='Id')

#1a. Seperate features and target from training dataset.
features = train_data.iloc[:,:-1]
target = train_data.loc[:, ['SalePrice']]
print('Train Set Size : ', train_data.shape)
print('Test Set Size : ', test_data.shape)
print('Train Features Size : ', features.shape)
num_train_rows = train_data.shape[0]
num_test_rows = test_data.shape[0]
print('Train Rows : ', num_train_rows)
print('Test Rows : ', num_test_rows)

#1b. Merge training and test datasets to cover all 
#encodings for categorical features
all_data = pd.concat((features, test_data)).reset_index(drop=True)

Train Set Size :  (1460, 80)
Test Set Size :  (1459, 79)
Train Features Size :  (1460, 79)
Train Rows :  1460
Test Rows :  1459


In [3]:
'''
2. In Numerical feature analysis, I found that SalPrice is NOT NORMALLY DISTRIBUTED.
So, apply LOG TRANSFORMATION to bring SalePrice closer to Normal Distribution.
'''
print('Skewness of SalePrice before Log Transform : %f'% target.skew())
print('Kurtosis of SalePrice before Log Transform : %f'% target.kurt())

'''
Skew = 1.882876 indicates positive skew with tail to the right.
Kurt = 6.536282 indicates heavy tails i.e. more data on tails.
'''

#Apply Log transformation
target['SalePrice'] = np.log(target['SalePrice'])
print('Skewness of SalePrice after Log Transform : %f'% target.skew())
print('Kurtosis of SalePrice after Log Transform : %f'% target.kurt())

Skewness of SalePrice before Log Transform : 1.882876
Kurtosis of SalePrice before Log Transform : 6.536282
Skewness of SalePrice after Log Transform : 0.121335
Kurtosis of SalePrice after Log Transform : 0.809532


In [4]:
#3. Missing Data
null_features = all_data.columns[all_data.isnull().any()]
missing_ratio = (all_data[null_features].isnull().sum()/len(all_data)) * 100
missing_data = pd.DataFrame({'Missing Ratio' :missing_ratio})
print(missing_data.sort_values(by='Missing Ratio',ascending=False))


for col in ['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu']:
    all_data[col] = all_data[col].fillna('None')

for col in ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'MasVnrType']:
    all_data[col] = all_data[col].fillna('None')
    
for col in ['BsmtExposure', 'BsmtFinType2', 'BsmtQual', 'BsmtCond', 'BsmtFinType1']:
    all_data[col] = all_data[col].fillna('None')

'''
No GarageYrBlt means no Garage. I can impute mean/median since it would 
incorrectly convey existence of Garage. same reasoning for MasVnrArea.
'''
for col in ['GarageYrBlt', 'MasVnrArea']:
    all_data[col] = all_data[col].fillna(0)

'''
Group data by neighborhood & imputed null LotFrontage columns with median of
grouped data.
'''
all_data['LotFrontage'] = all_data.groupby(['Neighborhood'])\
                    ['LotFrontage'].transform(lambda x : x.fillna(x.median()))
    
all_data['Electrical'] = \
    all_data['Electrical'].fillna(all_data['Electrical'].mode()[0])
    
all_data['MSZoning'] = \
    all_data['MSZoning'].fillna(all_data['MSZoning'].mode()[0])
    
all_data['Utilities'] = all_data['Utilities'].fillna('ELO')

all_data['Exterior1st'] = all_data['Exterior1st'].fillna('Other')

all_data['Exterior2nd'] = all_data['Exterior2nd'].fillna('Other')

all_data['SaleType'] = all_data['SaleType'].fillna('Oth')

all_data['Functional'] = \
    all_data['Functional'].fillna(all_data['Functional'].mode()[0])

all_data['KitchenQual'] = \
    all_data['KitchenQual'].fillna(all_data['KitchenQual'].mode()[0])

for col in ['BsmtFullBath', 'BsmtHalfBath', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'GarageCars', 'GarageArea']:
    all_data[col] = all_data[col].fillna(0)
    
null_features = all_data.columns[all_data.isnull().any()]
missing_ratio = (all_data[null_features].isnull().sum()/len(all_data)) * 100
missing_data = pd.DataFrame({'Missing Ratio' :missing_ratio})
print(missing_data.sort_values(by='Missing Ratio',ascending=False))

              Missing Ratio
PoolQC                99.66
MiscFeature           96.40
Alley                 93.22
Fence                 80.44
FireplaceQu           48.65
LotFrontage           16.65
GarageFinish           5.45
GarageQual             5.45
GarageCond             5.45
GarageYrBlt            5.45
GarageType             5.38
BsmtExposure           2.81
BsmtCond               2.81
BsmtQual               2.77
BsmtFinType2           2.74
BsmtFinType1           2.71
MasVnrType             0.82
MasVnrArea             0.79
MSZoning               0.14
BsmtFullBath           0.07
BsmtHalfBath           0.07
Functional             0.07
Utilities              0.07
GarageArea             0.03
GarageCars             0.03
Electrical             0.03
KitchenQual            0.03
TotalBsmtSF            0.03
BsmtUnfSF              0.03
BsmtFinSF2             0.03
BsmtFinSF1             0.03
Exterior2nd            0.03
Exterior1st            0.03
SaleType               0.03
Empty DataFrame
Colu

In [5]:
'''
                    4.Numerical Features - Analysis
                    
Linear models assume normal distribution of the features. So, lets check if they 
follow normal distribution or apply transformation to correct them.

Selecting following features based on analysis 
from BackwardElimination_NumericFeatureOnly.ipynb
'''
print('Skewness of OverallQual before Log Transform : %f'% all_data['OverallQual'].skew())
print('Kurtosis of OverallQual before Log Transform : %f'% all_data['OverallQual'].kurt())

print('Skewness of GarageCars before Log Transform : %f'% all_data['GarageCars'].skew())
print('Kurtosis of GarageCars before Log Transform : %f'% all_data['GarageCars'].kurt())

print('Skewness of YearBuilt before Log Transform : %f'% all_data['YearBuilt'].skew())
print('Kurtosis of YearBuilt before Log Transform : %f'% all_data['YearBuilt'].kurt())

print('Skewness of FullBath before Log Transform : %f'% all_data['FullBath'].skew())
print('Kurtosis of FullBath before Log Transform : %f'% all_data['FullBath'].kurt())

print('Skewness of TotalBsmtSF before Log Transform : %f'% all_data['TotalBsmtSF'].skew())
print('Kurtosis of TotalBsmtSF before Log Transform : %f'% all_data['TotalBsmtSF'].kurt())

print('Skewness of YearRemodAdd before Log Transform : %f'% all_data['YearRemodAdd'].skew())
print('Kurtosis of YearRemodAdd before Log Transform : %f'% all_data['YearRemodAdd'].kurt())

print('Skewness of TotRmsAbvGrd before Log Transform : %f'% all_data['TotRmsAbvGrd'].skew())
print('Kurtosis of TotRmsAbvGrd before Log Transform : %f'% all_data['TotRmsAbvGrd'].kurt())

print('Skewness of Fireplaces before Log Transform : %f'% all_data['Fireplaces'].skew())
print('Kurtosis of Fireplaces before Log Transform : %f'% all_data['Fireplaces'].kurt())

print('Skewness of LotArea before Log Transform : %f'% all_data['LotArea'].skew())
print('Kurtosis of LotArea before Log Transform : %f'% all_data['LotArea'].kurt())

print('Skewness of MSSubClass before Log Transform : %f'% all_data['MSSubClass'].skew())
print('Kurtosis of MSSubClass before Log Transform : %f'% all_data['MSSubClass'].kurt())

print('Skewness of OverallCond before Log Transform : %f'% all_data['OverallCond'].skew())
print('Kurtosis of OverallCond before Log Transform : %f'% all_data['OverallCond'].kurt())

print('Skewness of BsmtFinSF1 before Log Transform : %f'% all_data['BsmtFinSF1'].skew())
print('Kurtosis of BsmtFinSF1 before Log Transform : %f'% all_data['BsmtFinSF1'].kurt())

print('Skewness of EnclosedPorch before Log Transform : %f'% all_data['EnclosedPorch'].skew())
print('Kurtosis of EnclosedPorch before Log Transform : %f'% all_data['EnclosedPorch'].kurt())

print('Skewness of ScreenPorch before Log Transform : %f'% all_data['ScreenPorch'].skew())
print('Kurtosis of ScreenPorch before Log Transform : %f'% all_data['ScreenPorch'].kurt())

print('Skewness of YrSold before Log Transform : %f'% all_data['YrSold'].skew())
print('Kurtosis of YrSold before Log Transform : %f'% all_data['YrSold'].kurt())

print('Skewness of GrLivArea before Log Transform : %f'% all_data['GrLivArea'].skew())
print('Kurtosis of GrLivArea before Log Transform : %f'% all_data['GrLivArea'].kurt())

print('Skewness of KitchenAbvGr before Log Transform : %f'% all_data['KitchenAbvGr'].skew())
print('Kurtosis of KitchenAbvGr before Log Transform : %f'% all_data['KitchenAbvGr'].kurt())

print('Skewness of WoodDeckSF before Log Transform : %f'% all_data['WoodDeckSF'].skew())
print('Kurtosis of WoodDeckSF before Log Transform : %f'% all_data['WoodDeckSF'].kurt())

print('Skewness of BsmtFullBath before Log Transform : %f'% all_data['BsmtFullBath'].skew())
print('Kurtosis of BsmtFullBath before Log Transform : %f'% all_data['BsmtFullBath'].kurt())

Skewness of OverallQual before Log Transform : 0.197212
Kurtosis of OverallQual before Log Transform : 0.067219
Skewness of GarageCars before Log Transform : -0.219694
Kurtosis of GarageCars before Log Transform : 0.236592
Skewness of YearBuilt before Log Transform : -0.600114
Kurtosis of YearBuilt before Log Transform : -0.511317
Skewness of FullBath before Log Transform : 0.167692
Kurtosis of FullBath before Log Transform : -0.538129
Skewness of TotalBsmtSF before Log Transform : 1.157489
Kurtosis of TotalBsmtSF before Log Transform : 9.122827
Skewness of YearRemodAdd before Log Transform : -0.451252
Kurtosis of YearRemodAdd before Log Transform : -1.346431
Skewness of TotRmsAbvGrd before Log Transform : 0.758757
Kurtosis of TotRmsAbvGrd before Log Transform : 1.169064
Skewness of Fireplaces before Log Transform : 0.733872
Kurtosis of Fireplaces before Log Transform : 0.076424
Skewness of LotArea before Log Transform : 12.829025
Kurtosis of LotArea before Log Transform : 264.952310
S

In [6]:
'''
MSSubClass, OverallCond, BsmtFinSF1, EnclosedPorch, ScreenPorch, 
PoolArea, GrLivArea, KitchenAbvGr, WoodDeckSF
These features has high Skewness & Kurtosis
'''
TotalBsmtSFMean = all_data['TotalBsmtSF'].mean()
all_data.loc[all_data['TotalBsmtSF'] == 0, 'TotalBsmtSF'] = np.round(TotalBsmtSFMean).astype(int)
all_data['TotalBsmtSF'] = np.log(all_data['TotalBsmtSF'])
print('Skewness of TotalBsmtSF after Log Transform : %f'% all_data['TotalBsmtSF'].skew())
print('Kurtosis of TotalBsmtSF after Log Transform : %f'% all_data['TotalBsmtSF'].kurt())

OpenPorchSFMean = all_data['OpenPorchSF'].mean()
all_data.loc[all_data['OpenPorchSF'] == 0, 'OpenPorchSF'] = np.round(OpenPorchSFMean).astype(int)
all_data['TotalBsmtSF'] = np.log(all_data['TotalBsmtSF'])
print('Skewness of TotalBsmtSF after Log Transform : %f'% all_data['TotalBsmtSF'].skew())
print('Kurtosis of TotalBsmtSF after Log Transform : %f'% all_data['TotalBsmtSF'].kurt())

all_data['LotArea'] = np.log(all_data['LotArea'])
print('Skewness of LotArea after Log Transform : %f'% all_data['LotArea'].skew())
print('Kurtosis of LotArea after Log Transform : %f'% all_data['LotArea'].kurt())

all_data['MSSubClass'] = np.log(all_data['MSSubClass'])
print('Skewness of MSSubClass after Log Transform : %f'% all_data['MSSubClass'].skew())
print('Kurtosis of MSSubClass after Log Transform : %f'% all_data['MSSubClass'].kurt())

all_data['OverallCond'] = np.log(all_data['OverallCond'])
print('Skewness of OverallCond after Log Transform : %f'% all_data['OverallCond'].skew())
print('Kurtosis of OverallCond after Log Transform : %f'% all_data['OverallCond'].kurt())

all_data['GrLivArea'] = np.log(all_data['GrLivArea'])
print('Skewness of GrLivArea after Log Transform : %f'% all_data['GrLivArea'].skew())
print('Kurtosis of GrLivArea after Log Transform : %f'% all_data['GrLivArea'].kurt())

BsmtSF1Mean = all_data['BsmtFinSF1'].mean()
all_data.loc[all_data['BsmtFinSF1'] == 0, 'BsmtFinSF1'] = np.round(BsmtSF1Mean).astype(int)
all_data['BsmtFinSF1'] = np.log(all_data['BsmtFinSF1'])
print('Skewness of BsmtFinSF1 after Log Transform : %f'% all_data['BsmtFinSF1'].skew())
print('Kurtosis of BsmtFinSF1 after Log Transform : %f'% all_data['BsmtFinSF1'].kurt())

EnclosedPorchMean = all_data['EnclosedPorch'].mean()
all_data.loc[all_data['EnclosedPorch'] == 0, 'EnclosedPorch'] = np.round(EnclosedPorchMean).astype(int)
all_data['EnclosedPorch'] = np.log(all_data['EnclosedPorch'])
print('Skewness of EnclosedPorch after Log Transform : %f'% all_data['EnclosedPorch'].skew())
print('Kurtosis of EnclosedPorch after Log Transform : %f'% all_data['EnclosedPorch'].kurt())

ScreenPorchMean = all_data['ScreenPorch'].mean()
all_data.loc[all_data['ScreenPorch'] == 0, 'ScreenPorch'] = np.round(ScreenPorchMean).astype(int)
all_data['ScreenPorch'] = np.log(all_data['ScreenPorch'])
print('Skewness of ScreenPorch after Log Transform : %f'% all_data['ScreenPorch'].skew())
print('Kurtosis of ScreenPorch after Log Transform : %f'% all_data['ScreenPorch'].kurt())

KitchenAbvGrMean = all_data['KitchenAbvGr'].mean()
all_data.loc[all_data['KitchenAbvGr'] == 0, 'KitchenAbvGr'] = np.round(KitchenAbvGrMean).astype(int)
all_data['KitchenAbvGr'] = np.log(all_data['KitchenAbvGr'])
print('Skewness of KitchenAbvGr after Log Transform : %f'% all_data['KitchenAbvGr'].skew())
print('Kurtosis of KitchenAbvGr after Log Transform : %f'% all_data['KitchenAbvGr'].kurt())

WoodDeckSFMean = all_data['WoodDeckSF'].mean()
all_data.loc[all_data['WoodDeckSF'] == 0, 'WoodDeckSF'] = np.round(WoodDeckSFMean).astype(int)
all_data['WoodDeckSF'] = np.log(all_data['WoodDeckSF'])
print('Skewness of WoodDeckSF after Log Transform : %f'% all_data['WoodDeckSF'].skew())
print('Kurtosis of WoodDeckSF after Log Transform : %f'% all_data['WoodDeckSF'].kurt())

Skewness of TotalBsmtSF after Log Transform : -0.424181
Kurtosis of TotalBsmtSF after Log Transform : 1.712836
Skewness of TotalBsmtSF after Log Transform : -0.742638
Kurtosis of TotalBsmtSF after Log Transform : 2.775776
Skewness of LotArea after Log Transform : -0.505542
Kurtosis of LotArea after Log Transform : 3.754157
Skewness of MSSubClass after Log Transform : 0.229245
Kurtosis of MSSubClass after Log Transform : -1.145834
Skewness of OverallCond after Log Transform : -1.408074
Kurtosis of OverallCond after Log Transform : 11.127618
Skewness of GrLivArea after Log Transform : 0.012386
Kurtosis of GrLivArea after Log Transform : 0.212402
Skewness of BsmtFinSF1 after Log Transform : -1.845559
Kurtosis of BsmtFinSF1 after Log Transform : 6.243265
Skewness of EnclosedPorch after Log Transform : 2.342653
Kurtosis of EnclosedPorch after Log Transform : 4.062502
Skewness of ScreenPorch after Log Transform : 3.057237
Kurtosis of ScreenPorch after Log Transform : 7.673697
Skewness of Kit

In [7]:
'''
            5. LinearRegression Model
'''
final_features = all_data.loc[:,['MSSubClass', 'LotArea', 'OverallQual', 
                                     'OverallCond', 'YearBuilt', 'YearRemodAdd', 
                                       'BsmtFinSF1', 'TotalBsmtSF', 'GrLivArea', 
                                     'BsmtFullBath', 'FullBath', 'KitchenAbvGr', 
                                       'TotRmsAbvGrd', 'Fireplaces', 'GarageCars', 
                                     'WoodDeckSF', 'EnclosedPorch', 
                               'ScreenPorch',  'YrSold']]

train_selected_features = final_features[:num_train_rows]
test_selected_features = final_features[num_train_rows:]

from sklearn.cross_validation import train_test_split
X_train , X_test, y_train, y_test = train_test_split(train_selected_features, target, test_size=0.4, random_state=0)

from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)
y_train_predict = regressor.predict(X_test)
print("LinearRegression R-squared --> ", regressor.score(X_test, y_test)*100)

LinearRegression R-squared -->  87.67028488430935


c:\users\revan\appdata\local\programs\python\python36\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [8]:
'''
            6. LASSO Regression with K-Fold
'''
from sklearn.linear_model import Lasso, LassoCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import KFold, cross_val_predict, cross_val_score

kf = KFold(n_splits=5, shuffle=True, random_state=0)
lasso = make_pipeline(RobustScaler(), LassoCV(alphas=[0.002], random_state=0, cv=kf))
lasso.fit(X_train, y_train)
print("LASSO Regression R-squared --> ", lasso.score(X_test, y_test)*100)

LASSO Regression R-squared -->  87.52486439090472


c:\users\revan\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\coordinate_descent.py:1094: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [9]:
'''
            7. RIDGE Regression with K-Fold
'''
from sklearn.linear_model import Ridge, RidgeCV

kf = KFold(n_splits=5, shuffle=True, random_state=0)
ridge = make_pipeline(RobustScaler(), RidgeCV(alphas=[2.5], cv=kf))
ridge.fit(X_train, y_train)
print("RIDGE Regression R-squared --> ", ridge.score(X_test, y_test)*100)

RIDGE Regression R-squared -->  87.70744067789894


In [10]:
'''
            8. ElasticNet Regression with K-Fold
'''
from sklearn.linear_model import ElasticNet, ElasticNetCV

kf = KFold(n_splits=5, shuffle=True, random_state=0)
ENet = make_pipeline(RobustScaler(), ElasticNetCV(alphas=[0.003], cv=kf, random_state=0))
ENet.fit(X_train, y_train)
print("ElasticNet R-squared --> ", ENet.score(X_test, y_test)*100)

ElasticNet R-squared -->  87.58773038070525


c:\users\revan\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\coordinate_descent.py:1094: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [11]:
'''
            9. RANDOM FOREST Model
'''
from sklearn.ensemble import RandomForestRegressor
forest_regr = RandomForestRegressor(random_state=0, n_estimators=70,
                        min_samples_split=2, min_samples_leaf=1, max_features='sqrt',
                        max_depth=10, bootstrap=False)
forest_regr.fit(X_train, y_train)
print("Random Forest R-squared --> ", forest_regr.score(X_test, y_test)*100)

Random Forest R-squared -->  88.62647758122908


c:\users\revan\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


In [12]:
'''
            10. Code that calculates simple average of 4 model scores
'''
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
class AverageModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
        
    # we define clones of the original models to fit the data in
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]
        # Train cloned base models
        for model in self.models_:
            model.fit(X, y)
        return self
    
    #Now we do the predictions for cloned models and average them
    def predict(self, X):
        predictions = np.column_stack([
            model.predict(X) for model in self.models_
        ])
        return np.mean(predictions, axis=1)

In [14]:
'''
            11.Simple Average of 4 Models 
'''
average_models = make_pipeline(RobustScaler(), AverageModels(models = (ENet, ridge, lasso, forest_regr)))
average_models.fit(X_train, y_train)
y_predict = average_models.predict(X_test)
print("Simple Average of 4 Models R-squared --> ", average_models.score(X_test, y_test)*100)

from sklearn import metrics
print('MSE of average of 4 Models',metrics.mean_squared_error(y_test, y_predict))

Simple Average of 4 Models R-squared -->  89.02307039389127
MSE of average of 4 Models 0.016577311719483807


c:\users\revan\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\coordinate_descent.py:1094: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\revan\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\coordinate_descent.py:1094: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\revan\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:14: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


In [15]:
'''
            12. Stacked Model code
'''

class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=0)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y.iloc[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred.flatten()
                
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)

In [16]:
'''
            13. Stacked Model
'''
from sklearn import metrics

stacked_averaged_models = make_pipeline(RobustScaler(), \
                StackingAveragedModels(base_models = (ENet, lasso, forest_regr), meta_model = ridge))
X_train , X_test, y_train, y_test = train_test_split(train_selected_features, target, test_size=0.4, random_state=0)
stacked_averaged_models.fit(X_train, y_train)
stacked_train_pred = stacked_averaged_models.predict(X_test)
print("Stacked Model R-squared --> ", metrics.r2_score(y_pred=stacked_train_pred, y_true=y_test))
print('MSE of Stacked Model',metrics.mean_squared_error(y_test, stacked_train_pred))

c:\users\revan\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\coordinate_descent.py:1094: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\revan\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\coordinate_descent.py:1094: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\revan\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\coordinate_descent.py:1094: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\revan\appdata\local\programs\python\python36\lib\site-package

Stacked Model R-squared -->  0.8885636917904103
MSE of Stacked Model 0.016829063174740595


c:\users\revan\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:24: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
